<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Semiveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna
!pip install LAMDA-SSL

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.8/240.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
import sklearn.model_selection
from statistics import mean

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from LAMDA_SSL.Algorithm.Regression.CoReg import CoReg

import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

Evalueringsmetrikker

In [4]:
from sklearn.metrics import (mean_absolute_error,
                             mean_squared_error,
                             mean_absolute_percentage_error,
                             r2_score)

# Lager en funksjon for RMSE
def rmse(y_faktisk, y_predikert):
  return np.sqrt(mean_squared_error(y_faktisk, y_predikert))

# Algoritme: SSL Regression Heat Diffusion on graphs

In [5]:
from scipy import sparse
#from gbssl import LGC,HMN,PARW,OMNI,CAMLP
import random
import numpy as np
from sklearn.neighbors import kneighbors_graph

from sklearn.preprocessing import normalize
from scipy.io import loadmat

from sklearn.metrics.pairwise import rbf_kernel
from scipy.sparse import lil_matrix
from scipy import linalg
from sklearn.metrics import log_loss

from sklearn.neighbors import KNeighborsRegressor
from scipy.sparse.linalg import spsolve
from scipy.sparse import identity
import math
import warnings
warnings.filterwarnings("ignore")

In [6]:
def get_graph_matrix(X, k):

    # KNN graph
    KNN_graf = kneighbors_graph(X,
                                n_neighbors=k,
                                mode ="connectivity",
                                include_self=False)

    # Sigma
    n_egenskaper = X.shape[1]
    sigma = 1.0/ n_egenskaper

    # Normalisering for å unnga bias
    #distanser = np.sqrt((KNN_graf ** 2).sum(axis=1))
    #normalisert_KNN_graf = KNN_graf / distanser[: , np.newaxis]

    # RBF funksjon for vekter
    RBF_vekter = rbf_kernel(X, X, gamma=1.0 / (2 * sigma ** 2))

    KNN_RBF_graf = sparse.csr_matrix.multiply(KNN_graf, RBF_vekter)

    return KNN_RBF_graf

def get_harmonic_scores(G,treningsnode_markert_id, treningsnode_umarkert_id,testnode_id,yl):

    ##create Propagation Matrix###

    n = G.shape[0]
    graph = G.copy()
    degrees = graph.sum(axis=0).A[0]
    degrees[degrees==0] += 1  # Avoid division by 0
    D = sparse.diags((1.0/degrees),offsets=0)
    P = D.dot(graph).tolil() # Transition matrix
    P[treningsnode_markert_id] = 0
    all_values = np.zeros(n)
    all_values[treningsnode_markert_id] = 1

    all_values[treningsnode_umarkert_id] = 0
    all_values[testnode_id] = 0
    state_vector = all_values.copy()


    ###Propagate the scores####
    remaining_iter = 30
    state = state_vector.copy()
    Base = state.copy() # Initialize U = Y
    P = P.A

    while remaining_iter > 0:
        state = P.dot(state) + Base # Y^(k+1) <- Y^(k+1) + U
        remaining_iter -= 1

    state = np.round_(state, decimals=2)

    return state


def bhd(X_data, y_data, treningsnoder_markert, treningsnoder_umarkert, testnoder, k, alpha=1.0, maks_iterasjoner=100):

  graph_data = get_graph_matrix(X_data, k)
  y_trening_markert = y_data[treningsnoder_markert]

  state_hmn = get_harmonic_scores(graph_data,treningsnoder_markert, treningsnoder_umarkert, testnoder, y_trening_markert)

  graph = graph_data.copy()
  n = graph.shape[0]
  all_values = y_data.copy() # label vector
  all_values[testnoder] = np.mean(all_values[treningsnoder_markert]) #We imputed the values for the inlabeled nodes as the means of the labeled nodes
  all_values[treningsnoder_umarkert] = np.mean(all_values[treningsnoder_markert])
  remaining_iter = maks_iterasjoner

  I = np.eye(n,n,dtype=np.float64) # Identity matrix
  C = all_values - state_hmn # Constant C
  state = C.copy() # state vector
  state = state.reshape(n,1)
  L = sparse.csgraph.laplacian(graph,normed=True) # Laplacian matrix
  V = I + (-alpha/remaining_iter) * L

  while remaining_iter > 0:
      state = V.dot(state)
      remaining_iter -= 1

  state = state_hmn.reshape(n,1) + state

  y_prediksjon = state

  y_trening_prediksjon = np.ravel(y_prediksjon[treningsnoder_markert])
  y_test_prediksjon = np.ravel(y_prediksjon[testnoder])

  return y_test_prediksjon, y_trening_prediksjon

# Egendefinerte moduler

In [7]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [8]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import umarkert_andel_dag
from oppdeling_trening_testsett import enzymfordeling

In [9]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_semiveiledet.py /content/moduler_optimalisering_evaluering_semiveiledet.py

/content


In [10]:
from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_enzym
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_enzym

from moduler_optimalisering_evaluering_semiveiledet import trening_coreg_dag
from moduler_optimalisering_evaluering_semiveiledet import trening_bhd_dag

# Importering av relevant data

In [11]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                   header=0,
                   sep=',',
                   index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                          header=0,
                          sep=',',
                          index_col=0)

data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                          header=0,
                          sep=',',
                          index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [12]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 29049 entries, 2022-10-31 17:37:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  29049 non-null  int64  
 1   EnzymeType_A2  29049 non-null  int64  
 2   EnzymeType_B   29049 non-null  int64  
 3   EnzymeType_C   29049 non-null  int64  
 4   EnzymeType_D   29049 non-null  int64  
 5   EnzymeType_E   29049 non-null  int64  
 6   RawMatPercent  29049 non-null  float64
 7   NIRfat         29049 non-null  float64
 8   NIRash         29049 non-null  float64
 9   NIRwater       29049 non-null  float64
 10  TT08           29049 non-null  float64
 11  TT20           29049 non-null  float64
 12  TT12           29049 non-null  float64
 13  Mw             460 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [14]:
data_normal.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 18542 entries, 2022-11-02 00:00:00 to 2023-06-14 01:05:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  18542 non-null  int64  
 1   EnzymeType_A2  18542 non-null  int64  
 2   EnzymeType_B   18542 non-null  int64  
 3   EnzymeType_C   18542 non-null  int64  
 4   EnzymeType_D   18542 non-null  int64  
 5   EnzymeType_E   18542 non-null  int64  
 6   RawMatPercent  18542 non-null  float64
 7   NIRfat         18542 non-null  float64
 8   NIRash         18542 non-null  float64
 9   NIRwater       18542 non-null  float64
 10  TT08           18542 non-null  float64
 11  TT20           18542 non-null  float64
 12  TT12           18542 non-null  float64
 13  Mw             124 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 2.1 MB


In [15]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10507 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10507 non-null  int64  
 1   EnzymeType_A2  10507 non-null  int64  
 2   EnzymeType_B   10507 non-null  int64  
 3   EnzymeType_C   10507 non-null  int64  
 4   EnzymeType_D   10507 non-null  int64  
 5   EnzymeType_E   10507 non-null  int64  
 6   RawMatPercent  10507 non-null  float64
 7   NIRfat         10507 non-null  float64
 8   NIRash         10507 non-null  float64
 9   NIRwater       10507 non-null  float64
 10  TT08           10507 non-null  float64
 11  TT20           10507 non-null  float64
 12  TT12           10507 non-null  float64
 13  Mw             336 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [17]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering
resultater_kolonner = ["Treningsandel",
                       "RMSE test",
                       "R2 test",
                       "MAE test",
                       "MAPE test",
                       "RMSE trening",
                       "R2 trening",
                       "MAE trening",
                       "MAPE trening"]

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Et testsett. Fordelt på enzymtype (ikke tatt hensyn til kontinuitet)

In [22]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

# Definerer testandel
test_andel = 0.2

# Definere andel av treningsett som skal benyttes
trenings_andel = 1.0


# Deler datasettet i trening og testsett
treningssett_markert, enzymtyper_treningssett_markert, testsett,\
emzymtyper_testsett,treningssett_umarkert, enzymtyper_treningssett_umarkert,\
treningssett =trening_testsett_oppdeling_enzym(rå_data,
                                               data,
                                               test_andel=test_andel)


#Oversikt over fordelingen av enzymtyper i trening og testsett
print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_treningssett_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(emzymtyper_testsett.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_treningssett_umarkert.value_counts())



# Treningssett i forklaringsvariabler og respons
X_test = testsett.iloc[:, :-1]
y_test = testsett.iloc[:, -1]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
A2    128
E      60
C      56
B      56
D      50
A1     18
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    32
E     15
C     14
B     14
D     13
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


### CoReg

In [23]:
# Modelloptimalisering
coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                   enzymtyper_treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt=splitt,
                                                   gjentagelser=gjentagelser,
                                                   n_runder=n_runder,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 14:16:01,475] A new study created in memory with name: no-name-a39305a7-0528-4fa1-a22e-6c8ef17eb913
[I 2024-03-20 14:17:49,459] Trial 0 finished with value: 620.2946092856374 and parameters: {'k1': 4, 'k2': 3, 'p1': 5, 'p2': 10, 'max_iters': 19, 'pool_size': 133}. Best is trial 0 with value: 620.2946092856374.
[I 2024-03-20 14:18:56,425] Trial 1 finished with value: 616.9427464122954 and parameters: {'k1': 1, 'k2': 9, 'p1': 8, 'p2': 7, 'max_iters': 10, 'pool_size': 122}. Best is trial 1 with value: 616.9427464122954.


{'k1': 1, 'k2': 9, 'p1': 8, 'p2': 7, 'max_iters': 10, 'pool_size': 122}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,593.840639,0.937408,419.212209,0.063894,253.586444,0.98884,178.5635,0.027128


### BHD

In [24]:
# BHD
bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                               enzymtyper_treningssett_markert,
                                               treningssett_umarkert,
                                               enzymtyper_treningssett_umarkert,
                                               treningssett,
                                               testsett,
                                               X_test,
                                               y_test,
                                               splitt,
                                               gjentagelser,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 14:19:13,794] A new study created in memory with name: no-name-5a70d6d4-0a6f-4b52-aa90-d2707da4d85b
[I 2024-03-20 14:21:02,173] Trial 0 finished with value: 2413.755793665969 and parameters: {'k': 5563, 'alpha': 0.1, 'maks_iterasjoner': 31}. Best is trial 0 with value: 2413.755793665969.
[I 2024-03-20 14:22:05,003] Trial 1 finished with value: 4266.144005238888 and parameters: {'k': 1221, 'alpha': 10, 'maks_iterasjoner': 77}. Best is trial 0 with value: 2413.755793665969.


{'k': 5563, 'alpha': 0.1, 'maks_iterasjoner': 31}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,2411.855502,-0.032472,2074.564066,0.302462,382.898371,0.974557,298.816982,0.042731


## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.5, 0.7]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                   enzymtyper_treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt=splitt,
                                                   gjentagelser=gjentagelser,
                                                   n_runder=n_runder,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-19 19:10:16,250] A new study created in memory with name: no-name-d5082504-b0e3-4491-a23f-f400ce3cfb7c


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:23,691] Trial 0 finished with value: 4.4196957334928495 and parameters: {'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}. Best is trial 0 with value: 4.4196957334928495.
[I 2024-03-19 19:10:27,655] A new study created in memory with name: no-name-0bc4fce8-9453-4093-b10b-035d8a0e8352


Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:31,232] Trial 0 finished with value: 4.9286611096899415 and parameters: {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}. Best is trial 0 with value: 4.9286611096899415.
[I 2024-03-19 19:10:32,613] A new study created in memory with name: no-name-ee9b2a82-2957-4996-bcb2-901959c8fd9d


Enzymtyper Fordeling: Treningssett markert andel: 0.9
A2    23
B     14
C     14
D     10
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-19 19:10:39,398] Trial 0 finished with value: 3.9044307959768414 and parameters: {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}. Best is trial 0 with value: 3.9044307959768414.


{'trenings_andel': [0.5, 0.7, 0.9], 'parametere': [{'k1': 9, 'k2': 4, 'p1': 10, 'p2': 9, 'max_iters': 19, 'pool_size': 14}, {'k1': 5, 'k2': 1, 'p1': 3, 'p2': 5, 'max_iters': 1, 'pool_size': 154}, {'k1': 6, 'k2': 8, 'p1': 4, 'p2': 8, 'max_iters': 18, 'pool_size': 18}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,3.808039,0.582720,2.817819,0.152346,2.697001,0.666389,2.147705,0.089039
1,0.7,4.614026,0.387388,3.850733,0.201283,1.555246,0.895427,1.250396,0.057692
2,0.9,4.171275,0.499317,3.346993,0.178940,2.965527,0.651220,2.376538,0.112246


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-19 20:10:25,479] A new study created in memory with name: no-name-7c6a7a85-f538-40f0-9cc2-71a63dd70649


Enzymtyper Fordeling: Treningssett markert andel: 0.5
A2    13
C      8
B      8
D      5
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:11:25,482] Trial 0 finished with value: 5.463257439018376 and parameters: {'k': 6520, 'alpha': 0.01, 'maks_iterasjoner': 59}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:12:20,756] Trial 1 finished with value: 9.440781297034448 and parameters: {'k': 6420, 'alpha': 1, 'maks_iterasjoner': 21}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:13:09,686] Trial 2 finished with value: 13.284018048511529 and parameters: {'k': 3291, 'alpha': 10, 'maks_iterasjoner': 41}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:03,260] Trial 3 finished with value: 9.38236450932272 and parameters: {'k': 5615, 'alpha': 1, 'maks_iterasjoner': 67}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:14:24,211] Trial 4 finished with value: 13.283227581124137 and parameters: {'k': 608, 'alpha': 10, 'maks_iterasjoner': 61}. Best is trial 0 with value: 5.463257439018376.
[I 2024-03-19 20:15:20,603] Trial 5 finished with value: 9.4

Enzymtyper Fordeling: Treningssett markert andel: 0.7
A2    18
C     11
B     11
D      8
A1     1
Name: EnzymeCode, dtype: int64


[I 2024-03-19 20:21:47,669] Trial 0 finished with value: 5.170170181366991 and parameters: {'k': 6810, 'alpha': 0.1, 'maks_iterasjoner': 124}. Best is trial 0 with value: 5.170170181366991.
[I 2024-03-19 20:22:46,339] Trial 1 finished with value: 5.170124841396653 and parameters: {'k': 2400, 'alpha': 0.1, 'maks_iterasjoner': 175}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:23:47,677] Trial 2 finished with value: 5.1702718989745815 and parameters: {'k': 6635, 'alpha': 0.1, 'maks_iterasjoner': 75}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:27,375] Trial 3 finished with value: 9.511713575788109 and parameters: {'k': 1496, 'alpha': 1, 'maks_iterasjoner': 142}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:24:57,188] Trial 4 finished with value: 25.524638276899225 and parameters: {'k': 1557, 'alpha': 10, 'maks_iterasjoner': 8}. Best is trial 1 with value: 5.170124841396653.
[I 2024-03-19 20:25:52,165] Trial 5 finished with valu

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k': 2851, 'alpha': 0.01, 'maks_iterasjoner': 177}, {'k': 931, 'alpha': 0.01, 'maks_iterasjoner': 198}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.082883,-0.064742,4.513696,0.247507,0.126221,0.999269,0.093372,0.003768
1,1.0,6.111859,-0.074909,4.537162,0.249573,0.132864,0.999237,0.100209,0.004191


## Alternativ 2: Et testsett fordelt på dag og kontinuitet

In [44]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

# Definerer trening og testandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_trening_markert = enzymfordeling(rå_data,
                                       trening_markert)
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)

print("Enzymtyper Fordeling: Trening markert")
print(enzym_trening_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzym_trening_umarkert.value_counts())


# umarkert andel til BHD

# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Trening markert
A2    134
E      60
C      55
B      54
D      50
A1     16
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    26
B     16
C     15
E     15
D     13
A1     6
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


### CoReg

In [45]:
# CoReg
coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                 trening_markert,
                                                 trening_umarkert,
                                                 trening,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 n_runder,
                                                 umarkert_andel=False,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


# Legger resulater til i datasett
coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 15:13:13,152] A new study created in memory with name: no-name-05d23ba7-2b0a-4370-b05e-080350b54379
[I 2024-03-20 15:14:41,745] Trial 0 finished with value: 731.5907887132644 and parameters: {'k1': 9, 'k2': 8, 'p1': 3, 'p2': 4, 'max_iters': 19, 'pool_size': 104}. Best is trial 0 with value: 731.5907887132644.
[I 2024-03-20 15:14:54,499] Trial 1 finished with value: 737.3472029101931 and parameters: {'k1': 9, 'k2': 8, 'p1': 7, 'p2': 4, 'max_iters': 20, 'pool_size': 12}. Best is trial 0 with value: 731.5907887132644.


{'k1': 9, 'k2': 8, 'p1': 3, 'p2': 4, 'max_iters': 19, 'pool_size': 104}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,643.105856,0.919619,455.856265,0.064193,484.583485,0.959844,336.021885,0.051232


### BHD

In [46]:
# BHD
bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                             trening_markert,
                                             trening_umarkert,
                                             test,
                                             X_test,
                                             y_test,
                                             splitt,
                                             n_runder,
                                             umarkert_andel=umarkert_andel,
                                             trenings_andel=trenings_andel,
                                             random_seed=random_seed)


# Legger resulater til i datasett
bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

# Beste parametere
print(parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

[I 2024-03-20 15:20:00,265] A new study created in memory with name: no-name-2e8a822d-3f1f-4acc-88cd-8b0bdb48580f
[I 2024-03-20 15:21:16,945] Trial 0 finished with value: 2127.2735641739373 and parameters: {'k': 7349, 'alpha': 0.1, 'maks_iterasjoner': 124}. Best is trial 0 with value: 2127.2735641739373.
[I 2024-03-20 15:21:36,495] Trial 1 finished with value: 2226.5800874480874 and parameters: {'k': 410, 'alpha': 0.01, 'maks_iterasjoner': 50}. Best is trial 0 with value: 2127.2735641739373.


{'k': 7349, 'alpha': 0.1, 'maks_iterasjoner': 124}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,2349.865838,-0.073186,2092.581741,0.2938,504.499843,0.956476,388.874244,0.05671


## Alternativ 2: Flere treningssandeler

In [18]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    26
B     16
C     15
E     15
D     13
A1     6
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
A2    17987
A1     2823
D      2193
B      2043
C      1846
E      1697
Name: EnzymeCode, dtype: int64


### CoReg

In [19]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 15:27:13,528] A new study created in memory with name: no-name-77d34647-443e-4bcb-b3ae-3ccce255c618


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    134
E      60
C      55
B      54
D      50
A1     16
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:27:34,253] Trial 0 finished with value: 703.1147383611417 and parameters: {'k1': 5, 'k2': 3, 'p1': 6, 'p2': 2, 'max_iters': 5, 'pool_size': 181}. Best is trial 0 with value: 703.1147383611417.
[I 2024-03-20 15:27:48,255] Trial 1 finished with value: 737.5078416020515 and parameters: {'k1': 7, 'k2': 9, 'p1': 2, 'p2': 2, 'max_iters': 2, 'pool_size': 188}. Best is trial 0 with value: 703.1147383611417.


{'trenings_andel': [1.0], 'parametere': [{'k1': 5, 'k2': 3, 'p1': 6, 'p2': 2, 'max_iters': 5, 'pool_size': 181}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,672.422568,0.912123,474.244982,0.065673,397.4104,0.972992,272.138045,0.040787


### BHD

In [20]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    134
E      60
C      55
B      54
D      50
A1     16
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:29:39,565] A new study created in memory with name: no-name-23a6465d-2d7f-45d9-8c0e-6ba0873cbb53
[I 2024-03-20 15:30:04,597] Trial 0 finished with value: 1.1451791792705249e+18 and parameters: {'k': 681, 'alpha': 0.1, 'maks_iterasjoner': 82}. Best is trial 0 with value: 1.1451791792705249e+18.
[I 2024-03-20 15:31:14,147] Trial 1 finished with value: 3512.093901806471 and parameters: {'k': 4832, 'alpha': 1, 'maks_iterasjoner': 158}. Best is trial 1 with value: 3512.093901806471.


{'trenings_andel': [1.0], 'parametere': [{'k': 4832, 'alpha': 1, 'maks_iterasjoner': 158}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,3271.078441,-1.079556,2677.910522,0.353199,3342.692043,-0.910737,2545.770197,0.372238


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [34]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [0.5, 0.7]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
E     15
C     14
B     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [29]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 14:36:51,540] A new study created in memory with name: no-name-46c6e7e9-5ce8-451a-9350-dc602b994cc4


Enzymtyper Fordeling: Treningssett markert andel: 0.5
E     30
C     27
B     26
D     24
A2    20
A1     7
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:36:56,681] Trial 0 finished with value: 754.0999352852565 and parameters: {'k1': 9, 'k2': 5, 'p1': 4, 'p2': 3, 'max_iters': 7, 'pool_size': 23}. Best is trial 0 with value: 754.0999352852565.
[I 2024-03-20 14:38:15,940] Trial 1 finished with value: 917.3474555418026 and parameters: {'k1': 8, 'k2': 10, 'p1': 10, 'p2': 5, 'max_iters': 13, 'pool_size': 151}. Best is trial 0 with value: 754.0999352852565.
[I 2024-03-20 14:38:44,750] Trial 2 finished with value: 589.4485411450266 and parameters: {'k1': 2, 'k2': 3, 'p1': 3, 'p2': 2, 'max_iters': 10, 'pool_size': 190}. Best is trial 2 with value: 589.4485411450266.
[I 2024-03-20 14:38:56,349] Trial 3 finished with value: 932.4952750245341 and parameters: {'k1': 10, 'k2': 7, 'p1': 7, 'p2': 2, 'max_iters': 5, 'pool_size': 60}. Best is trial 2 with value: 589.4485411450266.
[I 2024-03-20 14:39:50,512] Trial 4 finished with value: 603.1767649501334 and parameters: {'k1': 7, 'k2': 2, 'p1': 1, 'p2': 9, 'max_iters': 19, 'pool_size':

Enzymtyper Fordeling: Treningssett markert andel: 0.7
E     42
C     37
B     36
D     33
A2    29
A1    10
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:42:50,275] Trial 0 finished with value: 665.8046670646543 and parameters: {'k1': 5, 'k2': 7, 'p1': 1, 'p2': 9, 'max_iters': 14, 'pool_size': 96}. Best is trial 0 with value: 665.8046670646543.
[I 2024-03-20 14:43:38,674] Trial 1 finished with value: 717.0407015371393 and parameters: {'k1': 6, 'k2': 7, 'p1': 3, 'p2': 5, 'max_iters': 13, 'pool_size': 117}. Best is trial 0 with value: 665.8046670646543.
[I 2024-03-20 14:43:43,237] Trial 2 finished with value: 582.3618840009984 and parameters: {'k1': 4, 'k2': 1, 'p1': 2, 'p2': 8, 'max_iters': 2, 'pool_size': 155}. Best is trial 2 with value: 582.3618840009984.
[I 2024-03-20 14:44:33,958] Trial 3 finished with value: 729.6697211898048 and parameters: {'k1': 5, 'k2': 9, 'p1': 3, 'p2': 5, 'max_iters': 17, 'pool_size': 90}. Best is trial 2 with value: 582.3618840009984.
[I 2024-03-20 14:44:35,805] Trial 4 finished with value: 634.2679401998006 and parameters: {'k1': 1, 'k2': 1, 'p1': 6, 'p2': 5, 'max_iters': 20, 'pool_size': 1

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k1': 6, 'k2': 2, 'p1': 2, 'p2': 6, 'max_iters': 15, 'pool_size': 125}, {'k1': 4, 'k2': 1, 'p1': 2, 'p2': 8, 'max_iters': 2, 'pool_size': 155}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,537.457049,0.939780,398.957521,0.051056,269.368446,0.985024,195.877151,0.026440
1,0.7,546.198556,0.937805,398.214420,0.050920,204.641854,0.991941,138.068869,0.018532


### BHD

In [35]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 14:56:48,766] A new study created in memory with name: no-name-b5f6a5bc-b290-4efa-899c-7e99e5ed5ef7


Enzymtyper Fordeling: Treningssett markert andel: 0.5
E     30
C     27
B     26
D     24
A2    20
A1     7
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:56:52,328] Trial 0 finished with value: 4542.599637508608 and parameters: {'k': 436, 'alpha': 10, 'maks_iterasjoner': 9}. Best is trial 0 with value: 4542.599637508608.
[I 2024-03-20 14:56:57,607] Trial 1 finished with value: 2205.6015948495583 and parameters: {'k': 1932, 'alpha': 0.01, 'maks_iterasjoner': 18}. Best is trial 1 with value: 2205.6015948495583.
[I 2024-03-20 14:56:59,877] Trial 2 finished with value: 2243.146394874511 and parameters: {'k': 223, 'alpha': 0.1, 'maks_iterasjoner': 21}. Best is trial 1 with value: 2205.6015948495583.
[I 2024-03-20 14:57:06,989] Trial 3 finished with value: 3163.305548188382 and parameters: {'k': 2129, 'alpha': 1, 'maks_iterasjoner': 148}. Best is trial 1 with value: 2205.6015948495583.
[I 2024-03-20 14:57:13,961] Trial 4 finished with value: 2205.6013763837095 and parameters: {'k': 2002, 'alpha': 0.01, 'maks_iterasjoner': 195}. Best is trial 4 with value: 2205.6013763837095.
[I 2024-03-20 14:57:19,582] Trial 5 finished with v

Enzymtyper Fordeling: Treningssett markert andel: 0.7
E     42
C     37
B     36
D     33
A2    29
A1    10
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:57:50,278] Trial 0 finished with value: 3272.3207835831054 and parameters: {'k': 299, 'alpha': 1, 'maks_iterasjoner': 155}. Best is trial 0 with value: 3272.3207835831054.
[I 2024-03-20 14:57:54,967] Trial 1 finished with value: 3280.498005060782 and parameters: {'k': 1109, 'alpha': 1, 'maks_iterasjoner': 42}. Best is trial 0 with value: 3272.3207835831054.
[I 2024-03-20 14:58:00,581] Trial 2 finished with value: 3275.305244802297 and parameters: {'k': 2060, 'alpha': 1, 'maks_iterasjoner': 78}. Best is trial 0 with value: 3272.3207835831054.
[I 2024-03-20 14:58:05,637] Trial 3 finished with value: 2319.033417306449 and parameters: {'k': 302, 'alpha': 0.1, 'maks_iterasjoner': 116}. Best is trial 3 with value: 2319.033417306449.
[I 2024-03-20 14:58:09,649] Trial 4 finished with value: 4511.281843960607 and parameters: {'k': 213, 'alpha': 10, 'maks_iterasjoner': 181}. Best is trial 3 with value: 2319.033417306449.
[I 2024-03-20 14:58:16,161] Trial 5 finished with value: 4

{'trenings_andel': [0.5, 0.7], 'parametere': [{'k': 2002, 'alpha': 0.01, 'maks_iterasjoner': 195}, {'k': 302, 'alpha': 0.1, 'maks_iterasjoner': 116}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.5,2195.157597,-0.004585,1975.605843,0.286302,41.487431,0.999645,31.418913,0.004034
1,0.7,2249.331867,-0.054781,1993.303903,0.281625,408.400539,0.967905,306.822318,0.039703


## Alternativ 2: Flere treningssandeler

In [26]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [0.9, 1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
E     15
B     13
C     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [27]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 16:23:34,230] A new study created in memory with name: no-name-6af24d9f-dc18-4f29-ab54-f58a5cef0b86


Enzymtyper Fordeling: Treningssett markert andel: 0.9
E     54
C     48
B     47
D     44
A2    37
A1    13
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:24:47,273] Trial 0 finished with value: 685.4226157275973 and parameters: {'k1': 2, 'k2': 9, 'p1': 1, 'p2': 2, 'max_iters': 20, 'pool_size': 137}. Best is trial 0 with value: 685.4226157275973.
[I 2024-03-20 16:25:27,526] Trial 1 finished with value: 676.5745106788513 and parameters: {'k1': 2, 'k2': 7, 'p1': 9, 'p2': 10, 'max_iters': 8, 'pool_size': 199}. Best is trial 1 with value: 676.5745106788513.
[I 2024-03-20 16:25:47,237] A new study created in memory with name: no-name-06c7cbaf-11fd-4162-8604-5afdbe9f900c


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     54
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:25:55,109] Trial 0 finished with value: 719.7342544291657 and parameters: {'k1': 5, 'k2': 2, 'p1': 7, 'p2': 2, 'max_iters': 4, 'pool_size': 85}. Best is trial 0 with value: 719.7342544291657.
[I 2024-03-20 16:26:29,874] Trial 1 finished with value: 760.380620936038 and parameters: {'k1': 8, 'k2': 7, 'p1': 7, 'p2': 5, 'max_iters': 11, 'pool_size': 84}. Best is trial 0 with value: 719.7342544291657.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'k1': 2, 'k2': 7, 'p1': 9, 'p2': 10, 'max_iters': 8, 'pool_size': 199}, {'k1': 5, 'k2': 2, 'p1': 7, 'p2': 2, 'max_iters': 4, 'pool_size': 85}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,646.036112,0.901551,475.760757,0.062065,319.314916,0.981575,212.332258,0.027493
1,1.0,653.485454,0.899267,465.878939,0.062051,324.776339,0.980357,215.058032,0.027707


### BHD

In [28]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 0.9
E     54
C     48
B     47
D     44
A2    37
A1    13
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:27:05,839] A new study created in memory with name: no-name-18845ae7-0f05-4c07-882d-7ac39baf6d5e
[I 2024-03-20 16:27:16,517] Trial 0 finished with value: 3794.304678758939 and parameters: {'k': 2709, 'alpha': 1, 'maks_iterasjoner': 159}. Best is trial 0 with value: 3794.304678758939.
[I 2024-03-20 16:27:23,048] Trial 1 finished with value: 2117.626867989527 and parameters: {'k': 1904, 'alpha': 0.01, 'maks_iterasjoner': 54}. Best is trial 1 with value: 2117.626867989527.


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     54
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-20 16:27:26,908] A new study created in memory with name: no-name-31192d15-b677-4b83-ac2c-b098c7d9d152
[I 2024-03-20 16:27:34,562] Trial 0 finished with value: 2146.203555625426 and parameters: {'k': 543, 'alpha': 0.01, 'maks_iterasjoner': 63}. Best is trial 0 with value: 2146.203555625426.
[I 2024-03-20 16:27:40,702] Trial 1 finished with value: 2093.801867623694 and parameters: {'k': 436, 'alpha': 0.1, 'maks_iterasjoner': 4}. Best is trial 1 with value: 2093.801867623694.


{'trenings_andel': [0.9, 1.0], 'parametere': [{'k': 1904, 'alpha': 0.01, 'maks_iterasjoner': 54}, {'k': 436, 'alpha': 0.1, 'maks_iterasjoner': 4}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,0.9,2066.984628,-0.007795,1874.909713,0.258751,52.637357,0.999499,36.485546,0.004684
1,1.0,2113.782221,-0.053946,1902.787039,0.254758,516.151119,0.950386,366.656682,0.047451


# NIR målinger ekskludert

In [21]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Flere treningsandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
A2    7
C     4
B     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                     enzymtyper_treningssett_markert,
                                                     treningssett_umarkert,
                                                     treningssett,
                                                     X_test,
                                                     y_test,
                                                     splitt=splitt,
                                                     gjentagelser=gjentagelser,
                                                     n_runder=n_runder,
                                                     trenings_andel=trenings_andel,
                                                     random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 13:41:56,278] A new study created in memory with name: no-name-43635a3c-b79b-4d96-8b8d-bc029bfc873f


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:42:09,724] Trial 0 finished with value: 5.050093033762192 and parameters: {'k1': 8, 'k2': 9, 'p1': 4, 'p2': 10, 'max_iters': 20, 'pool_size': 12}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:02,888] Trial 1 finished with value: 5.073281057754931 and parameters: {'k1': 5, 'k2': 10, 'p1': 4, 'p2': 7, 'max_iters': 17, 'pool_size': 65}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:19,090] Trial 2 finished with value: 5.399002928636507 and parameters: {'k1': 10, 'k2': 2, 'p1': 10, 'p2': 3, 'max_iters': 18, 'pool_size': 27}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:38,410] Trial 3 finished with value: 5.260300386083797 and parameters: {'k1': 10, 'k2': 2, 'p1': 6, 'p2': 6, 'max_iters': 2, 'pool_size': 148}. Best is trial 0 with value: 5.050093033762192.
[I 2024-03-20 13:43:41,041] Trial 4 finished with value: 4.916827370608138 and parameters: {'k1': 9, 'k2': 7, 'p1': 5, 'p2': 3, 'max_iters': 5, 'pool_size'

{'trenings_andel': [1.0], 'parametere': [{'k1': 6, 'k2': 9, 'p1': 8, 'p2': 9, 'max_iters': 1, 'pool_size': 34}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.942628,0.297023,4.232296,0.222778,3.937273,0.388229,3.213241,0.152856


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 13:46:46,443] A new study created in memory with name: no-name-bca6e7c0-187c-4bea-bce9-1a587c77e6b0


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    26
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:47:56,537] Trial 0 finished with value: 5.027610544334957 and parameters: {'k': 5051, 'alpha': 0.1, 'maks_iterasjoner': 146}. Best is trial 0 with value: 5.027610544334957.
[I 2024-03-20 13:48:49,062] Trial 1 finished with value: 5.027608547256884 and parameters: {'k': 2715, 'alpha': 0.1, 'maks_iterasjoner': 176}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:49:25,751] Trial 2 finished with value: 6.2877293088414685 and parameters: {'k': 1881, 'alpha': 1, 'maks_iterasjoner': 89}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:50:32,392] Trial 3 finished with value: 6.285797254057214 and parameters: {'k': 5743, 'alpha': 1, 'maks_iterasjoner': 128}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:51:00,931] Trial 4 finished with value: 9.354421732500974 and parameters: {'k': 1092, 'alpha': 10, 'maks_iterasjoner': 96}. Best is trial 1 with value: 5.027608547256884.
[I 2024-03-20 13:52:00,997] Trial 5 finished with value:

{'trenings_andel': [1.0], 'parametere': [{'k': 751, 'alpha': 0.01, 'maks_iterasjoner': 126}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,5.99464,-0.034074,4.49963,0.241266,0.072208,0.999794,0.059269,0.002705


## Alternativ 2: Flere treningssandeler

In [ ]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
A2    5
B     4
C     4
D     3
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert andel: 1.0
A2    18114
A1     2843
D      2242
B      2093
C      1896
E      1772
Name: EnzymeCode, dtype: int64


### CoReg

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 13:54:14,192] A new study created in memory with name: no-name-ca9e0276-d8ce-43a3-a29e-868aa4cf440c


Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:54:25,918] Trial 0 finished with value: 5.946402474305579 and parameters: {'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}. Best is trial 0 with value: 5.946402474305579.
[I 2024-03-20 13:56:00,212] Trial 1 finished with value: 6.0440976840206515 and parameters: {'k1': 6, 'k2': 4, 'p1': 5, 'p2': 10, 'max_iters': 20, 'pool_size': 186}. Best is trial 0 with value: 5.946402474305579.


{'trenings_andel': [1.0], 'parametere': [{'k1': 10, 'k2': 6, 'p1': 8, 'p2': 3, 'max_iters': 2, 'pool_size': 154}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.687323,0.069403,4.022689,0.196577,3.958656,0.440659,3.245401,0.158872


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 13:56:08,112] A new study created in memory with name: no-name-6a664a9a-e232-41ee-8994-ea442fbf3316
[I 2024-03-20 13:56:56,654] Trial 0 finished with value: 13.230903344108082 and parameters: {'k': 2315, 'alpha': 10, 'maks_iterasjoner': 178}. Best is trial 0 with value: 13.230903344108082.
[I 2024-03-20 13:57:24,290] Trial 1 finished with value: 8.072900120392212 and parameters: {'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}. Best is trial 1 with value: 8.072900120392212.


{'trenings_andel': [1.0], 'parametere': [{'k': 372, 'alpha': 1, 'maks_iterasjoner': 200}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,6.973406,-1.059689,5.683795,0.255547,8.019235,-1.295342,6.235015,0.266181


## Designproduksjon

## Alternativ 1: Flere treningsandeler

In [22]:
# Dataframes for lagring av resultater
coreg_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_enzym = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Setter fast testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert,test, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

print("Enzymtyper Fordeling: Trening markert")
print(enzymtyper_markert.value_counts())
print("Enzymtyper Fordeling: Test")
print(enzymtyper_test.value_counts())
print("Enzymtyper Fordeling: Trening umarkert")
print(enzymtyper_umarkert.value_counts())

# Treningssett
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]


# Treningsandeler
treningsandeler = [1.0]

# Umarkert andel til BDH
umarkert_andel = 0.25

# Kryssvalidering
splitt = 2
gjentagelser = 1

# Antall runder til optimalisering
n_runder = 10

Enzymtyper Fordeling: Trening markert
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Test
E     15
C     14
B     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Trening umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [23]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # Modelloptimalisering
  coreg_resultater, parametere = trening_coreg_enzym(treningssett_markert,
                                                    enzymtyper_treningssett_markert,
                                                    treningssett_umarkert,
                                                    treningssett,
                                                    X_test,
                                                    y_test,
                                                    splitt=splitt,
                                                    gjentagelser=gjentagelser,
                                                    n_runder=n_runder,
                                                    trenings_andel=trenings_andel,
                                                    random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_enzym.loc[len(coreg_resultater_enzym)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_enzym

[I 2024-03-20 15:52:44,688] A new study created in memory with name: no-name-c5592913-f6af-46e8-8ffa-00b8151b5ed4


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     53
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[W 2024-03-20 15:52:52,884] Trial 0 failed with parameters: {'k1': 9, 'k2': 7, 'p1': 1, 'p2': 2, 'max_iters': 6, 'pool_size': 192} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/content/moduler_optimalisering_evaluering_semiveiledet.py", line 276, in objective
    coreg.fit(X_trening_fold_markert_transformert,
  File "/usr/local/lib/python3.10/dist-packages/LAMDA_SSL/Algorithm/Regression/CoReg.py", line 79, in fit
    h_temp.predict(L_X[idx_o].reshape(1, -1))) ** 2
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_regression.py", line 236, in predict
    neigh_ind = self.kneighbors(X, return_distance=False)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_base.py", line 784, in kneighbors
    check_is_fitted(self)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

KeyboardInterrupt: 

### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzymtyper_treningssett_markert.value_counts())


  # BHD
  bhd_resultater, parametere = trening_bhd_enzym(treningssett_markert,
                                                 enzymtyper_treningssett_markert,
                                                 treningssett_umarkert,
                                                 enzymtyper_umarkert,
                                                 treningssett,
                                                 test,
                                                 X_test,
                                                 y_test,
                                                 splitt,
                                                 gjentagelser,
                                                 n_runder,
                                                 umarkert_andel=umarkert_andel,
                                                 trenings_andel=trenings_andel,
                                                 random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_enzym.loc[len(bhd_resultater_enzym)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_enzym

[I 2024-03-20 14:00:21,938] A new study created in memory with name: no-name-91250566-f45f-4c07-a896-bf4b2b4e11ea


Enzymtyper Fordeling: Treningssett markert andel: 1.0
C     14
B     14
D     11
A2     5
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:00:30,633] Trial 0 finished with value: 7.0477430036266195 and parameters: {'k': 2322, 'alpha': 1, 'maks_iterasjoner': 144}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:36,993] Trial 1 finished with value: 9.928171835120885 and parameters: {'k': 1405, 'alpha': 10, 'maks_iterasjoner': 113}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:43,501] Trial 2 finished with value: 7.4522799199520735 and parameters: {'k': 2582, 'alpha': 1, 'maks_iterasjoner': 3}. Best is trial 0 with value: 7.0477430036266195.
[I 2024-03-20 14:00:50,428] Trial 3 finished with value: 5.085764895284852 and parameters: {'k': 1909, 'alpha': 0.1, 'maks_iterasjoner': 93}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:00:59,400] Trial 4 finished with value: 9.92754531897237 and parameters: {'k': 2198, 'alpha': 10, 'maks_iterasjoner': 147}. Best is trial 3 with value: 5.085764895284852.
[I 2024-03-20 14:01:02,658] Trial 5 finished with value

{'trenings_andel': [1.0], 'parametere': [{'k': 490, 'alpha': 0.01, 'maks_iterasjoner': 21}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.988597,-0.204528,4.089666,0.206132,0.091019,0.999674,0.066162,0.003041


## Alternativ 2: Flere treningssandeler

In [24]:
# Dataframes for lagring av resultater
coreg_resultater_dag = pd.DataFrame(columns = resultater_kolonner)
bhd_resultater_dag = pd.DataFrame(columns = resultater_kolonner)

coreg_beste_parametere = {'trenings_andel': [],
                          'parametere': []}

bhd_beste_parametere = {'trenings_andel': [],
                        'parametere': []}

# Definerer testandel
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, test,\
trening_umarkert, trening = trening_testsett_oppdeling_dag(rå_data,
                                                           data_design_uten_NIR,
                                                           test_andel=test_andel,
                                                           trening_krav=True)


#Oversikt over fordelingen av enzymtyper i trening og testsett
enzym_test = enzymfordeling(rå_data,
                            test)
enzym_trening_umarkert = enzymfordeling(rå_data,
                                        trening_umarkert)



print("Enzymtyper Fordeling: Test")
print(enzym_test.value_counts())
print("Enzymtyper Fordeling: Treningssett umarkert")
print(enzym_trening_umarkert.value_counts())


# Treningssett i forklaringsvariabler og respons
X_test = test.iloc[:, :-1]
y_test = test.iloc[:, -1]

# Treningsandeler
trenings_andeler = [1.0]

# Umarkert andel for bhd
umarkert_andel = 0.25

# Parametervalg for kryssvalidering
splitt = 2

# Antall runder til optimalisering
n_runder = 2

Enzymtyper Fordeling: Test
E     15
B     13
C     13
D     12
A2    10
A1     4
Name: EnzymeCode, dtype: int64
Enzymtyper Fordeling: Treningssett umarkert
D     1926
A1    1826
B     1728
C     1710
E     1631
A2    1350
Name: EnzymeCode, dtype: int64


### CoReg

In [25]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # CoReg
  coreg_resultater, parametere = trening_coreg_dag(rå_data,
                                                   treningssett_markert,
                                                   treningssett_umarkert,
                                                   treningssett,
                                                   X_test,
                                                   y_test,
                                                   splitt,
                                                   n_runder,
                                                   umarkert_andel=False,
                                                   trenings_andel=trenings_andel,
                                                   random_seed=random_seed)


  # Legger resulater til i datasett
  coreg_resultater_dag.loc[len(coreg_resultater_dag)] = coreg_resultater

  coreg_beste_parametere['trenings_andel'].append(trenings_andel)
  coreg_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(coreg_beste_parametere)

# Data med resultater fra treningsandeler
coreg_resultater_dag

[I 2024-03-20 15:53:01,605] A new study created in memory with name: no-name-d08ab546-a594-4b92-b2ad-2bf98f0fa81f


Enzymtyper Fordeling: Treningssett markert andel: 1.0
E     60
C     54
B     52
D     48
A2    41
A1    14
Name: EnzymeCode, dtype: int64


[I 2024-03-20 15:53:14,966] Trial 0 finished with value: 1286.9719016355389 and parameters: {'k1': 9, 'k2': 8, 'p1': 4, 'p2': 3, 'max_iters': 11, 'pool_size': 27}. Best is trial 0 with value: 1286.9719016355389.
[I 2024-03-20 15:54:08,287] Trial 1 finished with value: 1355.1537948613854 and parameters: {'k1': 8, 'k2': 2, 'p1': 1, 'p2': 1, 'max_iters': 13, 'pool_size': 160}. Best is trial 0 with value: 1286.9719016355389.


{'trenings_andel': [1.0], 'parametere': [{'k1': 9, 'k2': 8, 'p1': 4, 'p2': 3, 'max_iters': 11, 'pool_size': 27}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,649.19508,0.900586,502.535489,0.067652,835.132393,0.870115,636.376849,0.086558


### BHD

In [ ]:
# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       trening_krav=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert.value_counts())

  # BHD
  bhd_resultater, parametere = trening_bhd_dag(rå_data,
                                               treningssett_markert,
                                               treningssett_umarkert,
                                               test,
                                               X_test,
                                               y_test,
                                               splitt,
                                               n_runder,
                                               umarkert_andel=umarkert_andel,
                                               trenings_andel=trenings_andel,
                                               random_seed=random_seed)


  # Legger resulater til i datasett
  bhd_resultater_dag.loc[len(bhd_resultater_dag)] = bhd_resultater

  bhd_beste_parametere['trenings_andel'].append(trenings_andel)
  bhd_beste_parametere['parametere'].append(parametere)

# Beste parametere
print(bhd_beste_parametere)

# Data med resultater fra treningsandeler
bhd_resultater_dag

Enzymtyper Fordeling: Treningssett markert andel: 1.0
A2    28
B     16
C     16
D     11
A1     2
Name: EnzymeCode, dtype: int64


[I 2024-03-20 14:02:33,802] A new study created in memory with name: no-name-2637f184-92b8-481b-b30b-a576cf014d62
[I 2024-03-20 14:03:06,355] Trial 0 finished with value: 11.875057535216705 and parameters: {'k': 1786, 'alpha': 1, 'maks_iterasjoner': 2}. Best is trial 0 with value: 11.875057535216705.
[I 2024-03-20 14:04:00,607] Trial 1 finished with value: 6.394147053894678 and parameters: {'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}. Best is trial 1 with value: 6.394147053894678.


{'trenings_andel': [1.0], 'parametere': [{'k': 2593, 'alpha': 0.01, 'maks_iterasjoner': 162}]}


,Treningsandel,RMSE test,R2 test,MAE test,MAPE test,RMSE trening,R2 trening,MAE trening,MAPE trening
0,1.0,4.95906,-0.041623,4.439962,0.21455,0.173033,0.998931,0.141838,0.006167
